In [ ]:
from pyspark.sql import SparkSession
import sys, os
script_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "Scripts"))
if script_dir not in sys.path:
    sys.path.insert(0, script_dir)
from load_table_script import load_table_with_logging

In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [ ]:
load_table_with_logging("DS.MD_CURRENCY_D", "/data/md_currency_d.csv", spark)